# Building A Convolutional Image Classifier With Keras and Tensorflow

This project revolves around using modern deep-learning networks to build an image classifier with Keras. We will design our own custom convnet with reusable blocks and perform visual feature extraction. We will also use transfer learning to boost our model and utilize data augmentation to extend our dataset. 

Before we begin, let's break down the theory behind our project so we get a better understanding. 

The goal of our project is to design a neural network which can "understand" a natural image well-enough to solve the same kinds of problems the human visual system can solve. 

There are many neural networks (eg. RNNs, GNNs, CNNs), all utilized for different purposes and applications in machine learning. For example, Recurrent Neural Networks (RNNs) are great for text based classification tasks. This can include objectives like sentiment analysis. The neural networks that are best for image classification are called convolutional neural networks (CNN or convnet). 

A CNN consists of two parts: a convolutional base and a dense head. 

The base is used to extract the features from an image. What does this mean? Each convolutional layer applies filters (small matrices) that detect specific patterns in different part of the image. These filters help break down the image into different levels of abstraction. 
- First layers detect basic features (edges, corners, textures). 
- Middle layers detect more complex structures (shapes, object). 
- Deeper layers recognize high-level features (eg. faces, cats, cars).

Each layer transforms the image into multiple feature maps, which are "filtered versions" of the image, highlighting different aspects. By gradually learning from low-level details to high-level concepts, the CNN builds an abstract understanding of the image inputted allowing the classification to become easier. 

The head now recieves meaningful structured information from the base instead of raw data/pixels. This allows it to better make an educated guess/classificaiton when given data. 

Now, during training, we want our network to learn two things. 
1. which features to extract from an image 
2. which class goes with what features

CNNs are rarely trained from scratch and a more common approach is to reuse the base of a pretrained model. To the pretrained base, we can then attach an untrained head. In other wrods, we reuse the part of a network that has already learned to extract features and attach it to some fresh layers to learn. 

Enough talking, let's get coding! 

## Step 1 - Loading the Data

In [8]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
%pip install kagglehub
import kagglehub

# Reproducibility
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells

# Download the dataset using KaggleHub API
path = kagglehub.dataset_download("ryanholbrook/car-or-truck")

# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    os.path.join(path, 'train'),
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    os.path.join(path, 'valid'),
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


Note: you may need to restart the kernel to use updated packages.


/opt/miniconda3/envs/cnnproj/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 5117 files belonging to 2 classes.


2025-01-29 20:22:22.246947: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 5051 files belonging to 2 classes.


## Step 2 - Define Pretrained Base 

The most commonly used dataset for pretraining is ImageNet, a large dataset of many kind of natural images. Keras, which is a deep learning framework contians a multitude of pre-trained models -- many of them being trained on ImageNet. One of these pretrained model is called VGG16 which is the model we will use for our project. 

Some information abotu VGG16: 

Architecture:

VGG16 consists of 16 layers in total, which include:
- 13 convolutional layers: These layers apply filters to the input image to detect features like edges, textures, and patterns.
- 3 fully connected layers: After extracting features through convolution, these layers help the network make decisions or classifications based on those features.
It uses a relatively simple and uniform structure where each convolutional layer uses 3x3 filters and 2x2 max-pooling layers. This helps the model focus on learning spatial hierarchies of features in images.

What this means is that after the 13 convolutional layers have extracted low-level features such as edges, textures, or complex patterns, the resulting feature maps are flattened into a one-dimensional vector. This vector is then passed to the fully connected layers otherwise known as dense layers. Fully connected layers use connections from the one-dimensional vectors to learn how to combine the low-level features into a high-level image such as a cat or a dog. 

Note: transforming the resulting features into a one-dimensional vector and passing the result into dense layers are not done in the base. They are performed in the head of the CNN. 

Pretrained on ImageNet:
- VGG16 is commonly used as a pre-trained model that has been trained on a large dataset called ImageNet. ImageNet consists of millions of labeled images across 1000 different categories (e.g., animals, objects, scenes).
Because of this, VGG16 has already learned to extract general features from images (e.g., edges, textures, shapes) and can be used for transfer learning, where you fine-tune it for a specific task (such as classifying new categories of images).

Why use VGG16?
- Transfer Learning: Instead of training a deep neural network from scratch (which can be computationally expensive), you can use VGG16 pretrained on ImageNet. This approach allows you to leverage the features the model has already learned, and you can fine-tune it to your own dataset. This is often faster and requires less data.

Performance: VGG16 has proven to be a very effective model for image classification and is used in many research and real-world applications.

In [9]:
from tensorflow.keras.applications import VGG16

# Load the VGG16 model pre-trained on ImageNet
pretrained_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(128, 128, 3)
)
pretrained_base.trainable = False

## Step 3 - Attach Head

In Convolutional Neural Networks (CNNs), "attaching the head" refers to adding the final layers that transform the features learned by the convolutional base (e.g., VGG16) into a final prediction (like "Truck" or "Car"). This process involves two key components: the Flatten layer and the Dense layers.

Let's break this down:

1. Flatten Layer:
Purpose: The Flatten layer is used to convert the 2D data (which is the output from the convolutional layers) into 1D data, so that it can be passed into the fully connected layers (the classifier "head").

Explanation: After the convolutional layers, the model has a multi-dimensional tensor (e.g., a matrix with width, height, and depth --> eg for color this could be red, green, and yellow). This tensor is the feature map containing the learned features. The Flatten layer simply flattens this 2D data into a 1D vector, which can be fed into the next fully connected layers.

Example: If the output of the last convolutional layer is a 2D matrix (e.g., shape = [batch_size, height, width, depth]), the Flatten layer will convert this into a 1D vector of length height * width * depth.

2. Dense Layer (Hidden Layer):
Purpose: The Dense layer, also known as a fully connected layer, is where the actual classification happens. It learns to combine the features extracted by the convolutional layers into higher-level concepts (e.g., "Is this a truck or a car?").

Explanation: A Dense layer connects every input from the previous layer (the flattened vector) to each of its output neurons. The first Dense layer, which is typically the hidden layer, has several neurons (units) that learn to recognize patterns in the data.

Example: If the input to this Dense layer is a vector of 512 values (after flattening), this layer might have 128 or 256 neurons that try to combine the features and extract higher-level patterns.

Each neuron focuses on a slightly different feature or has different weightings. So for example, a neuron might care more about the shape while the other one might care more abotu the color. As a result, the calculations and therefore the output of each neuron differs. This is why each input vector value must connect to EVERY single neuron in the hidden/dense layer. 

3. Output Layer (Final Dense Layer):
Purpose: The final Dense layer transforms the output from the previous Dense layer into a probability score, usually with respect to the target classes.

Explanation: This is the layer that makes the final classification decision. It often has one neuron for each class (for binary classification, like "Truck" or "Car", there will be one neuron). The output is passed through a softmax (or sigmoid) activation function, which turns the raw output into a probability score.

Example: For binary classification, the output neuron might give a value between 0 and 1. If it's a truck, the model will output something like 0.9 (indicating high confidence), and for a car, it might output 0.1.

The sigmoid function is often used to map raw values (called logits or scores) into probabilities. While sigmoid is great for binary classification, it is not the best choice for multi-class classification or for tasks where you need to output probabilities for more than two classes. In those cases, a more suitable option is the softmax activation function, which generalizes sigmoid to handle multiple classes.

Putting It All Together:
- Convolutional Base: This part extracts low-level and high-level features from the input image (such as edges, textures, and objects).
- Flatten: This layer converts the 2D feature maps into a 1D vector.
- Dense Layer (Hidden): This layer performs the classification by processing the features from the Flatten layer.
- Output Layer: The final Dense layer gives the prediction probabilities for each class (Truck or Car, for example).

In summary, the head of the model (Dense layers) takes the complex, high-level features extracted by the convolutional base (VGG16, in this case), and makes the final prediction about what the image represents (e.g., a truck, car, or other objects).

ReLU (Rectified Linear Unit) is an activation function commonly used in neural networks, defined as:
- For any input greater than 0, ReLU returns the input value.
- For any input less than or equal to 0, ReLU returns 0.

Why is ReLU Used?
- Non-Linearity: ReLU introduces non-linearity to the network, enabling it to learn complex patterns (as opposed to only linear relationships). By making every negative output 0, ReLU is saying "focus on only the important neurons (ie the positive ones).
- Sparsity: It makes the network "sparse" by setting negative values to 0, meaning fewer neurons are active at any given time. This helps focus learning on the most relevant features.
- Prevents Exploding Gradients: ReLU avoids the issue of large gradients (exploding gradients) that can destabilize training. This is only for negative numbers. 
- Faster Convergence: ReLU speeds up training by keeping gradients consistent (for positive values), leading to faster learning. Since it changes every negative value to 0, it allows the model to focus on fewer neurons, making it more efficient. 
- Avoids Saturation: 
    - Saturation refers to the point where the activation function's output becomes "flat" and no longer changes significantly with large input values. This happens with sigmoid and tanh functions since they are constrained between 0 and 1. 
    - Unlike sigmoid or tanh, ReLU doesn't saturate for positive values, allowing the network to learn efficiently without diminishing gradients.


In [10]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

## Step 4 - Train

Optimizer (adam):
- Adam (Adaptive Moment Estimation) is a popular optimization algorithm that adjusts learning rates for each parameter (each weight or bias, a neuron can have multiple weights) dynamically.
It helps speed up training and works well for many deep learning problems.

Loss Function (binary_crossentropy):
- Since this is a binary classification problem (only two possible classes), we use binary cross-entropy as the loss function.
- This measures how far the predicted probability is from the actual class (0 or 1).

Metrics (binary_accuracy):
- Tracks the percentage of correct predictions (i.e., how often the predicted class matches the actual class).

ds_train:
- The training dataset used to train the model.

validation_data=ds_valid:
- The validation dataset helps monitor the model’s performance on unseen data (not used in training). After every epoch, the model evaluates itself on teh validation dataset. If the accuracy for the training data increases every epoch while the accuracy for the validation data remians the same, then the model is overfitting. 
- NOTE: The model does not adjust its weights on the validation data. It simply uses its current parameters and weightings to obtain the accuracy. If it adjusted the weightings we would be "cheating" our model by making it learn from unseen data, resulting in overly-optimistic results.

epochs=30:
- The model will go through the dataset 30 times to learn from the data. 

verbose=0:
- Controls the output displayed during training.
- verbose=0 means no output is shown.
- verbose=1 would show progress bars.
- verbose=2 would show one line per epoch.


In [11]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=30,
    verbose=0,
)

2025-01-29 20:22:23.090115: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


When you call model.fit, it automatically returns history.history which is a dictionary that has 4 keys: 'loss,' 'val_loss,' 'binary_accuracy,' and 'val_binary_accuracy.' Loss refers to the error of the model while accuracy refers to the proportion of accurate predictions the model makes. In other words, loss and binary accuracy should be opposite (eg. when loss goes down, binary accuracy should go up and vice versa).

:, means "select all the rows" and ['loss', 'val_loss'] isolates the rows which in this case is 'loss' and 'val_loss' you want to select. 

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();